# **Initialization**

In [ ]:
# Global Vars
GENERATE_MATLAB_DATA = False
DATA_PATH = '/home/cyu/workspace/202312-1-Outcome-Prediction-and-Consciousness-Detection-in-Patients-With-Acute-TBI/data/mri_data_pandas.pkl'
RESULT_PATH = '/home/cyu/workspace/202312-1-Outcome-Prediction-and-Consciousness-Detection-in-Patients-With-Acute-TBI/data/MRI_Model.keras'

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
from keras.applications import ResNet50
from keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, GlobalAveragePooling3D, add, BatchNormalization, Activation, Dropout
from keras.models import Model
import keras
import scipy.io
from keras.regularizers import l2
from keras import backend as K
import tensorflow
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import numpy as np
import cv2
import pandas as pd

In [ ]:
# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("GPU Available:", gpu)
else:
    print("No GPU devices found.")

# **Load MRI Data from Matlab file (run once if data didn't change)**

In [ ]:
# Read in Matlab data
if GENERATE_MATLAB_DATA:
  from google.colab import drive
  drive.mount('/content/drive')

  matlab_datasets = []

  for i in range(1, 17):
    try:
      mat = scipy.io.loadmat('/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/image_data{num}.mat'.format(num = i))
      matlab_datasets.append(mat)
    except:
      continue

  # print(len(matlab_datasets))

  patient_ids = []
  designators = []
  imaging_types = []
  techniques = []
  datasets = []

  # create a Pandas dataframe
  for matfile in matlab_datasets:

    try:
      for line in matfile['image_data'][0]:
        patient_ids.append(line[0][0])
        designators.append(line[1][0])
        imaging_types.append(line[2][0])
        techniques.append(line[3][0])
        datasets.append(line[4])
    except:
      print('line issue: ', line[5][0])

  data = {
      "Patient_ID": patient_ids,
      "Designator": designators,
      "Imaging_Type": imaging_types,
      "Technique": techniques,
      "Data": datasets,
  }

  df = pd.DataFrame(data)
  # print(df)

  # save Pandas dataframe
  df.to_pickle(DATA_PATH)


# **Load MRI Data from Pandas DF (start here)**

In [ ]:
# load Pandas dataframe
if GENERATE_MATLAB_DATA:
    drive.mount('/content/drive')
file_name = DATA_PATH
df_loaded = pd.read_pickle(file_name)
# print(df_loaded)

# **Organize Data for Training**

In [ ]:
def get_patient_list(mat_file):
  """
  Input: mat_file - matlab file
  Output: patient_list - set of patient IDs
  """
  patient_list = []
  for line in mat_file['image_data'][0]:
    patient_list.append(line[0][0])

  patient_list = set(patient_list)

  return patient_list


def map_patient_to_img_technique(mat_file, patient_list):
  """
  Input: mat_file - matlab file
         patient_list - set or list of patients
  Output: patient_img_technique_map - dict of patients with their MRI image type
  """
  patient_img_technique_map = {}

  for patient_id in patient_list:
    img_techniques = []
    for line in mat_file['image_data'][0]:
      if line[0][0] == patient_id and line[3][0] not in img_techniques:
        img_techniques.append(line[3][0])
    patient_img_technique_map[patient_id] = img_techniques

  return patient_img_technique_map


def rescale_img(img_array, resolution):
  """
  Input: img_array - 2D numpy image array
         resolution - tuple of (height, width)
  Output: res - rescaled 2D numpy image array
  """
  if img_array.shape != resolution:
    res = cv2.resize(img_array, dsize=resolution, interpolation=cv2.INTER_CUBIC)
  else:
    res = img_array
  return res


def stack_mri_slices(img_slices):
  """

  """
  img_3D = np.dstack(img_slices)
  img_3D = img_3D[:, :, :, np.newaxis]
  return img_3D

# print(df_loaded['Technique'].value_counts())


In [ ]:
# rescale images to desired resolution
TARGET_RESOLUTION = (256, 256)
for idx in df_loaded.index:
  try:
    df_loaded['Data'][idx] = rescale_img(df_loaded['Data'][idx], TARGET_RESOLUTION)
    # print(df_loaded['Data'][idx].shape)
  except:
    print('failed at: ', idx)

In [ ]:
# get list of patients and shuffle

patient_list = df_loaded['Patient_ID'].unique() # get a list of patients
# print(patient_list)
patient_list = np.delete(patient_list, 58)      # remove a patient ID
patient_list = np.delete(patient_list, np.where(patient_list == '02445263'))
patient_list = np.delete(patient_list, np.where(patient_list == '15816944'))
# print(len(patient_list))
# print(patient_list)
np.random.shuffle(patient_list)                 # shuffle the patient list
# print(patient_list)

train_patients = patient_list[0:46]   # 70% of patients for training
# print(train_patients)
test_patients = patient_list[46:]     # 30% of patients for testing
# print(test_patients)

In [ ]:

df_loaded.loc[df_loaded['Patient_ID'] == '15816944', 'Technique']

In [ ]:
# build training/testing dataset

x_data_train_dict = {}
y_data_train_dict = {}
x_data_test_dict = {}
y_data_test_dict = {}
x_data_all_dict = {}
y_data_all_dict = {}

num_slices = 64

# TRAINING data x
for patient_id in train_patients:
  try:
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'SWAN']
    num_mri_slices = patient_mri_data['Data'].shape[0]
    # print(num_mri_slices)

    start_idx = int(num_mri_slices/2) - 32
    end_idx = int(num_mri_slices/2) +32
    print('start: ', start_idx, 'end: ', end_idx)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[start_idx:end_idx])
    # print(patient_mri_data.shape)
    x_data_train_dict[patient_id] = patient_mri_data

  except:
    print('no data: ', patient_id)
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'Ax DWI Asset']
    num_mri_slices = patient_mri_data['Data'].shape[0]
    # print(num_mri_slices)

    start_idx = int(num_mri_slices/2) - 32
    end_idx = int(num_mri_slices/2) + 32
    print('start: ', start_idx, 'end: ', end_idx)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[start_idx:end_idx])
    # print(patient_mri_data.shape)
    x_data_train_dict[patient_id] = patient_mri_data

# print(len(x_data_train_dict))
# print(x_data_train_dict)

# TRAINING data y
for patient_id in train_patients:
  outcome = df_loaded.loc[df_loaded['Patient_ID'] == patient_id, 'Designator'].iloc[0]
  if outcome == 'responsive':
    y_data_train_dict[patient_id] = 1
  elif outcome == 'unresponsive':
    y_data_train_dict[patient_id] = 0

# print(len(y_data_train_dict))
# print(y_data_train_dict)

# TESTING data x
# print(len(test_patients))
for patient_id in test_patients:
  # print(patient_id)
  try:
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'SWAN']
    num_mri_slices = patient_mri_data['Data'].shape[0]
    # print(num_mri_slices)

    start_idx = int(num_mri_slices/2) - 32
    end_idx = int(num_mri_slices/2) +32
    print('start: ', start_idx, 'end: ', end_idx)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[start_idx:end_idx])

    # print(patient_mri_data.shape)
    x_data_test_dict[patient_id] = patient_mri_data

  except:
    # print('no data: ', patient_id)
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'Ax DWI Asset']
    num_mri_slices = patient_mri_data['Data'].shape[0]
    # print(num_mri_slices)

    start_idx = int(num_mri_slices/2) - 32
    end_idx = int(num_mri_slices/2) +32
    print('start: ', start_idx, 'end: ', end_idx)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[start_idx:end_idx])
    # print(patient_mri_data.shape)
    x_data_test_dict[patient_id] = patient_mri_data

# print(len(x_data_test_dict))
# print(x_data_test_dict)


# TESTING data y
for patient_id in test_patients:
  outcome = df_loaded.loc[df_loaded['Patient_ID'] == patient_id, 'Designator'].iloc[0]
  if outcome == 'responsive':
    y_data_test_dict[patient_id] = 1
  elif outcome == 'unresponsive':
    y_data_test_dict[patient_id] = 0

#print(len(y_data_test_dict))
# print(y_data_train_dict)

# print(patient_mri_data.shape)
# print(patient_df['Technique'].value_counts())

In [ ]:
#print('x data train shape:')
#for patient in x_data_train_dict:
#  print(x_data_train_dict[patient].shape)

#print('x data test shape:')
#for patient in x_data_test_dict:
#  print(x_data_test_dict[patient].shape)

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for patient_id in train_patients:
  x_train.append(x_data_train_dict[patient_id])
  y_train.append(y_data_train_dict[patient_id])

for patient_id in test_patients:
  x_test.append(x_data_test_dict[patient_id])
  y_test.append(y_data_test_dict[patient_id])

x_train = np.asarray(x_train)
print(x_train.shape)
y_train = np.asarray(y_train)
# print(y_train.shape)
x_test = np.asarray(x_test)
# print(x_test.shape)
y_test = np.asarray(y_test)
# print(y_test.shape)

train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))

batch_size = 1
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    # .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 1
# Augment the on the fly during training.
test_dataset = (
    test_loader.shuffle(len(x_test))
    # .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

# 3D Resnet

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = Input((width, height, depth, 1))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = BatchNormalization()(x)

    # x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    # x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling3D()(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs, outputs, name="3dcnn")
    return model

# Build model.
WIDTH = 256
HEIGHT = 256
DEPTH = 64
model = get_model(width=WIDTH, height=HEIGHT, depth=DEPTH)
model.summary()

initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

In [ ]:
# Train model
train_out = model.fit(train_dataset, validation_data=test_dataset, epochs=50)

In [ ]:
# Save model
model.save(RESULT_PATH)

In [ ]:
import matplotlib.pyplot as plt

print(train_out.history.keys())

# summarize history for accuracy
plt.plot(train_out.history['acc'])
plt.plot(train_out.history['val_acc'])
plt.ylim([0, 1])
plt.title('MRI model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(train_out.history['loss'])
plt.plot(train_out.history['val_loss'])
plt.ylim([0, 1])
plt.title('MRI model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Load model
loaded_model = keras.saving.load_model(RESULT_PATH)

In [ ]:
model.predict(test_dataset.take(4))

In [ ]:
x_data_all_dict = {**x_data_train_dict, **x_data_test_dict}
y_data_all_dict = {**y_data_train_dict, **y_data_test_dict}

print('label:', y_data_all_dict['00123691'])

eval_data = x_data_all_dict['00123691']
eval_data = eval_data[None, :]
print(eval_data.shape)
eval_loader = tf.data.Dataset.from_tensors(eval_data)

loaded_model.predict(eval_loader)